In [1]:
from sympy import *
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
from IPython.display import display, Math, Latex
init_printing(use_latex='mathjax')
import numpy as np
import pandas as pd
from IPython.display import HTML, display
from tabulate import tabulate
from copy import deepcopy
from prettytable import PrettyTable

In [2]:
def print_eq(eq, *exprs, rev_int2=False):
    if type(eq) == str:
        eq = Symbol(eq)
    out = '$$ {} ' + '= {}' * len(exprs) + '$$'
    exprs=list(exprs)
    for i in range(len(exprs)):
        if type(exprs[i]) != str:
            exprs[i] = latex(exprs[i])
    if rev_int2:
        exprs[-2] += '|_0^x'
    display(Latex(out.format(latex(eq), *exprs)))

def to_s(i):
    return Symbol(str(i))

def print_system(*exprs, collab=False): 
    '''
    collab негодяй не хочет нормально системы уравнений отоброжать, 
    если вы в Jupyter, то замените в аргументе collab True на False'''
    exprs=list(exprs)
    for i in range(len(exprs)):
        if type(exprs[i]) != str:
            exprs[i] = latex(exprs[i])
    exprs_latex = r'\\'.join(exprs)
    if collab:
        system = r'Начало системы\\' + exprs_latex + r'\\Конец_системы\\'
    else:
        system = \
        r'''\begin{equation*}
    \begin{cases}''' \
        + exprs_latex + \
        r'''\end{cases}
    \end{equation*}
        '''

    print_eq(system)
    
def get_coef(el):
    args = el.args
    if len(args) == 0:
        return 1
    elif ask(Q.extended_real(args[0])):
        return int(args[0])
    
def print_table(array):
    pt = PrettyTable()
    pt.header = False
    pt.add_rows(array)
    print(pt)
    
def left_bas(values):
    table_bas = [[]] * (values.shape[0] - 1)
    for i_column in np.where(np.count_nonzero(values, axis=0) == 1)[0]:
        nz_el = values[:, i_column].max()
        i_nz_el = np.where(values[:, i_column] == nz_el)[0][0]
        table_bas[i_nz_el] = [x_dict[f'x{i_column + 1}'], f'x{i_column + 1}']
    return table_bas

def change_row(main_row, changed_row, null_i):
    factor = changed_row[null_i] / main_row[null_i]
    return changed_row - main_row * factor

def create_expr(coefs):
    expr = 0
    for i, coef in enumerate(coefs):
        expr += coef * to_s(f'x_{i + 1}')
    return expr

In [3]:
x1, x2, x3, x4 = symbols('x_1, x_2, x_3, x_4')
L = Symbol(r'L(\bar{x})')

# Вставьте сюда ваши параметры

In [4]:
L_coefs = [22, 31, 27] # сюда вставляется верхняя строчка таблицы

conss = [ # остальные строчки
    [2, 5, 9, 90],
    [14, 4, 3, 84]
]

L_expr = 0
for i, coef in enumerate(L_coefs):
    L_expr += coef * to_s(f'x_{i + 1}')
    
print_eq(L, latex(L_expr) + r'\rightarrow max')
conss_x_out = '' 
for i in range(1, len(L_coefs) + 1):
    conss_x_out += f' \ x_{{{i}}} \geqslant 0,'
print_eq(conss_x_out[:-1])

<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [5]:
can_conss = [*conss]
for i, coefs in enumerate(conss):
    add_coef = [0] * len(conss)
    add_coef[i] = 1
    can_conss[i] = Eq(create_expr(coefs[:-1] + add_coef), coefs[-1])
    
L_expr = create_expr(L_coefs)
x_values = L_coefs + [0] * (len(conss))
xs = [f'x{i}' for i in range(1, len(L_coefs) + len(conss) + 1)]
x_dict = {x: value for x, value in zip(xs, x_values)}
print('Приведём запись к каноническому виду')
print_eq(L, latex(L_expr) + r'\rightarrow max')
print('При ограничениях:')
print_system(*can_conss)
print_eq(r'x_{j} \geqslant 0,\ j=\overline{1,' + str(len(L_coefs) + len(conss)) + '}')

Приведём запись к каноническому виду


<IPython.core.display.Latex object>

При ограничениях:


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

In [6]:
def init_table(L_coefs, conss):
    conss = np.array(conss)
    table = []
    header = np.vstack((x_values, xs))
    header = np.hstack(([['ci', 'БП'], ['', '']], header, [['L(x)'], ['bi']]))
    table_coef_x = conss[:, :-1]
    table = np.hstack((table_coef_x, np.eye(len(conss), dtype=int)))
    bs = conss[:, -1]
    bs = bs.reshape((bs.size, 1))
    table = np.hstack((table, bs))
    footer = [coef * - 1 for coef in L_coefs] + [0] * (len(conss) + 1)
    table = np.vstack((table, footer))
    basises = left_bas(table)
    left_side = np.vstack((basises, [['', 'j']]))
    table = np.hstack((left_side, table))
    table = np.vstack((header, table)).astype(object)
    table[2:, 2:] = table[2:, 2:].astype(int)
    return table

def print_solution(table, n):
    use_basises = {basis: b for basis, b in zip(table[2:-1, 1], table[2:-1, -1])}
    solution_X = []
    for x in x_dict:
        if x in use_basises:
            solution_X.append(use_basises[x])
        else:
            solution_X.append(0)
    solution_X = tuple(solution_X)
    print_eq(f'\\bar X_{n}', solution_X)
    L_out = f'L(\\bar x_{{{n}}})'
    if n == 'опт':
        L_out = 'L(\\bar x)_{max}'
    print_eq(L_out, table[-1, -1])

def is_optimal(table):
    values = table[2:, 2:]
    return True if values[-1].min() >= 0 else False

def step_simplex_method(table):
    change_table = deepcopy(table)
    values = change_table[2:, 2:]
    per_col_i = np.where(values[-1] == values[-1].min())[0][0]
    per_col = values[:-1, per_col_i]
    bi = values[:-1, -1]
    per_bi = np.array([Rational(b, per_val) for b, per_val in zip(bi, per_col)])
    per_row_i = 0
    for i, b in enumerate(per_bi):
        if b >= 0 and b < per_bi[per_row_i]:
            per_row_i = i
    per_el = values[per_row_i, per_col_i]
    per_row = values[per_row_i, :] * Rational(1, per_el)
    values[per_row_i] = per_row
    for i in range(values.shape[0]):
        if i != per_row_i:
            values[i] = change_row(per_row, values[i], per_col_i)
    change_table[2:-1, :2] = left_bas(values)
    change_table[2:, 2:] = values
    return change_table

def simplex_method(L_coefs, conss):
    table = init_table(L_coefs, conss)
    print_table(table)
    n = 0
    while True:
        n += 1
        if is_optimal(table):
            print('В индексной строке нет отрицательных элементов. Найденное опорное решение является оптимальным')
            print_solution(table, 'опт')
            break
        else:
            print_solution(table, n)
            print('В индексной строке есть отрицательные элементы. Найденное решение не является оптимальным')
        table = step_simplex_method(table)
        print_table(table)
    return table
table = simplex_method(L_coefs, conss)

+----+----+-----+-----+-----+----+----+------+
| ci | БП |  22 |  31 |  27 | 0  | 0  | L(x) |
|    |    |  x1 |  x2 |  x3 | x4 | x5 |  bi  |
| 0  | x4 |  2  |  5  |  9  | 1  | 0  |  90  |
| 0  | x5 |  14 |  4  |  3  | 0  | 1  |  84  |
|    | j  | -22 | -31 | -27 | 0  | 0  |  0   |
+----+----+-----+-----+-----+----+----+------+


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

В индексной строке есть отрицательные элементы. Найденное решение не является оптимальным
+----+----+-------+----+-------+------+----+------+
| ci | БП |   22  | 31 |   27  |  0   | 0  | L(x) |
|    |    |   x1  | x2 |   x3  |  x4  | x5 |  bi  |
| 31 | x2 |  2/5  | 1  |  9/5  | 1/5  | 0  |  18  |
| 0  | x5 |  62/5 | 0  | -21/5 | -4/5 | 1  |  12  |
|    | j  | -48/5 | 0  | 144/5 | 31/5 | 0  | 558  |
+----+----+-------+----+-------+------+----+------+


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>

В индексной строке есть отрицательные элементы. Найденное решение не является оптимальным
+----+----+----+----+--------+--------+-------+----------+
| ci | БП | 22 | 31 |   27   |   0    |   0   |   L(x)   |
|    |    | x1 | x2 |   x3   |   x4   |   x5  |    bi    |
| 31 | x2 | 0  | 1  | 60/31  |  7/31  | -1/31 |  546/31  |
| 22 | x1 | 1  | 0  | -21/62 | -2/31  |  5/62 |  30/31   |
|    | j  | 0  | 0  | 792/31 | 173/31 | 24/31 | 17586/31 |
+----+----+----+----+--------+--------+-------+----------+
В индексной строке нет отрицательных элементов. Найденное опорное решение является оптимальным


<IPython.core.display.Latex object>

<IPython.core.display.Latex object>